# [STARTER] Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [1]:
# Only needed for Udacity workspace
"""
import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')
"""

'\nimport importlib.util\nimport sys\n\n# Check if \'pysqlite3\' is available before importing\nif importlib.util.find_spec("pysqlite3") is not None:\n    import pysqlite3\n    sys.modules[\'sqlite3\'] = sys.modules.pop(\'pysqlite3\')\n'

In [2]:
# TODO: Import the necessary libs
# For example: 
import os

from lib.agents import Agent
from lib.rag import RAG
from lib.llm import LLM
from lib.messages import UserMessage, SystemMessage, ToolMessage, AIMessage, BaseMessage
from lib.tooling import tool
from dotenv import load_dotenv
import chromadb
from chromadb.utils import embedding_functions
from lib.evaluation import TestCase, AgentEvaluator, EvaluationResult, EvaluationReport
from typing import List, Dict
from tavily import TavilyClient
from lib.parsers import PydanticOutputParser
from datetime import datetime

In [3]:
# TODO: Load environment variables
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

In [4]:
rag_llm = LLM(
    model="gpt-4o-mini",
    temperature=0.3,
)

### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [5]:
# TODO: Create retrieve_game tool
# It should use chroma client and collection you created
chroma_client = chromadb.PersistentClient(path="chromadb")

embedding_fn = embedding_functions.OpenAIEmbeddingFunction(
    api_key=OPENAI_API_KEY,
    api_base="https://openai.vocareum.com/v1")

collection = chroma_client.get_collection("udaplay", embedding_function=embedding_fn)
# Tool Docstring:
#    Semantic search: Finds most results in the vector DB
#    args:
#    - query: a question about game industry. 
#
#    You'll receive results as list. Each element contains:
#    - Platform: like Game Boy, Playstation 5, Xbox 360...)
#    - Name: Name of the Game
#    - YearOfRelease: Year when that game was released for that platform
#    - Description: Additional details about the game

retrieve_game_rag = RAG(
    llm=rag_llm,
    vector_store = collection
)
@tool
def retrieve_game(query):
    """
    Semantic search: Finds most results in the vector DB
    args:
    - query: a question about game industry. 
    """
    result:Run = retrieve_game_rag.invoke(query)
    return result.get_final_state()["answer"]

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given


#### Evaluate Retrieval Tool

In [6]:
# TODO: Create evaluate_retrieval tool
# You might use an LLM as judge in this tool to evaluate the performance
# You need to prompt that LLM with something like:
# "Your task is to evaluate if the documents are enough to respond the query. "
# "Give a detailed explanation, so it's possible to take an action to accept it or not."
# Use EvaluationReport to parse the result
# Tool Docstring:
#    Based on the user's question and on the list of retrieved documents, 
#    it will analyze the usability of the documents to respond to that question. 
#    args: 
#    - question: original question from user
#    - retrieved_docs: retrieved documents most similar to the user query in the Vector Database
#    The result includes:
#    - useful: whether the documents are useful to answer the question
#    - description: description about the evaluation result

@tool
def evaluate_retrieval(question: str, retrieved_docs: List[str]) -> EvaluationReport:
    """
    Based on the user's question and on the list of retrieved documents, 
    it will analyze the usability of the documents to respond to that question. 
    args: 
    - question: original question from user
    - retrieved_docs: retrieved documents most similar to the user query in the Vector Database
    The result includes:
    - useful: whether the documents are useful to answer the question
    - description: description about the evaluation result
    """
    llm_judge = LLM(model="gpt-4o-mini")
    docs_text = "\n\n".join(retrieved_docs) if retrieved_docs else "No documents provided."
    prompt = (
        "Your task is to evaluate if the documents are enough to respond the query.\n"
        f"Question: {question}\n"
        f"Documents:\n{docs_text}\n"
        "Give a detailed explanation, so it's possible to take an action to accept it or not."
    )

    response = llm_judge.invoke(prompt, response_format=EvaluationReport)
    parser = PydanticOutputParser(model_class=EvaluationReport)
    try:
        report = parser.parse(response)
    except Exception as e:
        report = EvaluationReport(useful=False, description=f"Failed to parse judge response: {e}")
    return report

#### Game Web Search Tool

In [7]:
# TODO: Create game_web_search tool
# Please use Tavily client to search the web
# Tool Docstring:
#    Semantic search: Finds most results in the vector DB
#    args:
#    - question: a question about game industry. 

@tool
def game_web_search(query: str) -> Dict:
    """
    Search the web using Tavily API
    args:
        query (str): Search query
    """
    client = TavilyClient(api_key=TAVILY_API_KEY)
    
    # Perform the search
    search_result = client.search(
        query=query,
        include_answer=True,
        include_raw_content=False,
        include_images=False
    )
    
    # Format the results
    formatted_results = {
        "answer": search_result.get("answer", ""),
        "results": search_result.get("results", []),
        "search_metadata": {
            "timestamp": datetime.now().isoformat(),
            "query": query
        }
    }
    
    return formatted_results

### Agent

In [8]:
# TODO: Create your Agent abstraction using StateMachine
# Equip with an appropriate model
# Craft a good set of instructions 
# Plug all Tools you developed


agent_instructions = """
You **must call `evaluate_retrieval` immediately after every `retrieve_game`** invocation:

1. Always run `retrieve_game` with the user’s question.
2. Regardless of what `retrieve_game` returns—even “I don’t know.” or an empty list—you **must** call `evaluate_retrieval` next.  
   - Pass the original user question and the retrieved documents (or an empty list).
   - Wait for the evaluation result before invoking any other tool.
3. Only after `evaluate_retrieval` completes may you decide whether another tool (e.g., `game_web_search`) is needed.  
   - If you plan to call `game_web_search`, explicitly justify why the evaluation deemed the retrieval insufficient.

Failure to call `evaluate_retrieval` after `retrieve_game` is a violation of these instructions. If you skip it, you must restart the sequence.
"""

udaplay_agent = Agent(
    model_name="gpt-4o-mini",
    instructions=agent_instructions,
    tools=[retrieve_game, evaluate_retrieval, game_web_search],
    temperature=0.3,
)


In [9]:
# TODO: Invoke your agent
# - When Pokémon Gold and Silver was released?
# - Which one was the first 3D platformer Mario game?
# - Was Mortal Kombat X realeased for Playstation 5?
def print_messages(messages: List[BaseMessage]):
    for m in messages:
        print(f" -> (role = {m.role}, content = {m.content}, tool_calls = {getattr(m, 'tool_calls', None)})")

In [10]:
run1 = udaplay_agent.invoke("When Pokémon Gold and Silver was released?", session_id="run1")
messages = run1.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Starting: __entry__


Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


[StateMachine] Executing step: retrieve
[StateMachine] Executing step: augment
[StateMachine] Executing step: generate
[StateMachine] Terminating: __termination__
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
 -> (role = system, content = 
You **must call `evaluate_retrieval` immediately after every `retrieve_game`** invocation:

1. Always run `retrieve_game` with the user’s question.
2. Regardless of what `retrieve_game` returns—even “I don’t know.” or an empty list—you **must** call `evaluate_retrieval` next.  
   - Pass the original user question and the retrieved documents (or an empty list).
   - Wait for the evaluation result before invoking any other tool.
3. Only after `evaluate_retrieval` completes may you decide whether another tool (e.g., `game_web_search`) is needed.  
   - If you plan to call `game

In [11]:
run2 = udaplay_agent.invoke("Which one was the first 3D platformer Mario game?", session_id="run2")
messages = run2.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Starting: __entry__
[StateMachine] Executing step: retrieve
[StateMachine] Executing step: augment
[StateMachine] Executing step: generate
[StateMachine] Terminating: __termination__
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
 -> (role = system, content = 
You **must call `evaluate_retrieval` immediately after every `retrieve_game`** invocation:

1. Always run `retrieve_game` with the user’s question.
2. Regardless of what `retrieve_game` returns—even “I don’t know.” or an empty list—you **must** call `evaluate_retrieval` next.  
   - Pass the original user question and the retrieved documents (or an empty list).
   - Wait for the evaluation result before invoking any oth

In [12]:
run3 = udaplay_agent.invoke("Was Mortal Kombat X realeased for Playstation 5?", session_id="run3")
messages = run3.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Starting: __entry__
[StateMachine] Executing step: retrieve
[StateMachine] Executing step: augment
[StateMachine] Executing step: generate
[StateMachine] Terminating: __termination__
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
 -> (role = system, content = 
You **must call `evaluate_retrieval` immediately after every `retrieve_game`** invocation:

1. Always run `retrieve_game` with the user’s question.
2. Regardless of what `retrieve_game` returns—even “I don’t know.” or an empty list—you **must** call `evaluate_retrieval` next.  
   - Pass the original user question and the retrieved

### (Optional) Advanced

In [13]:
# TODO: Update your agent with long-term memory
# TODO: Convert the agent to be a state machine, with the tools being pre-defined nodes